In [1]:
import wmfdata as wmf

Are there any countries which show up in the MaxMind geolocation data which aren't included in our `canonical_data.countries` dataset?

In [2]:
pageview_country = wmf.spark.run("""
SELECT DISTINCT
    continent,
    country_code,
    country
FROM wmf.pageview_hourly
WHERE
    year = 2022
    AND month = 11
""")

SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/15 03:11:32 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/12/15 03:11:32 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
22/12/15 03:11:32 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
22/12/15 03:11:32 WARN Utils: Service 'sparkDriver' could not bind on port 12002. Attempting port 12003.
22/12/15 03:11:32 WARN Utils: Service 'sparkDriver' could not bind on port 12003. Attempting port 12004.
22/12/15 03:11:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/15 03:11:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/12/15 03:11:33 WARN Utils: Service 'SparkUI' could n

In [3]:
canonical_data_country = wmf.spark.run("""
SELECT *
FROM canonical_data.countries
""")

In [9]:
merged = pageview_country.merge(
    right=canonical_data_country,
    how="outer",
    left_on="country_code",
    right_on="iso_code"
)

No, there's nothing missing from the canonical dataset. The four records which didn't match are all countries which are listed in the canonical dataset but didn't show up in the page view data stream in November 2022.

In [15]:
merged[merged.isnull().any(axis="columns")]

,continent,country_code,country,name,iso_code,economic_region,maxmind_continent
249,NaN,NaN,NaN,Antarctica,AQ,Global South,Antarctica
250,NaN,NaN,NaN,Bouvet Island,BV,Global North,Antarctica
251,NaN,NaN,NaN,Heard Island and McDonald Islands,HM,Global South,Antarctica
252,NaN,NaN,NaN,South Georgia and the South Sandwich Islands,GS,Global South,Antarctica


However, a meticulous look through the Wikipedia article [ISO 3166-1 alpha-2](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2) does show two name changes we should implement ("Swaziland" → "Eswatini", "Macedonia" → "North Macedonia") as well as one name we should change for clarity ("Caribbean Netherlands" → "Bonaire, Sint Eustatius, and Saba", since "Caribbean Netherlands" might be taken to include Aruba, Curaçao, and Sint Maarten).